In [83]:
import pandas as pd
import operator
import urllib
from wordcloud import WordCloud as wc
import numpy as np
import sklearn
import requests
import re

data = pd.read_csv("federal_sample_20170408.csv")
data = pd.DataFrame(data)

In [84]:
params = ['txt_link', 'alleged_pros_misconduct', 'misconduct_determination']
new = data[params]

misconduct_terms = [' no misconduct', 'REVIEW', 'harmless error/no misconduct',
 'harmless error/no misconduct?', 'harmless error/no misconduct??','motion denied' ,
 'no misconduct', 'no misconduct(see notes)', 'no misconduct/harmless error',
 'no misconduct/harmless error??', 'no misconduct/harmless error???','prejudicial', 'prejudicial ',
 'prejudicial error', 'prejudicial misconduct', 'procedurally barred','procedurally barred // no misconduct','remand for evidentiary hearing']

remove_terms = ['no determinatation','no determination', 'na','no ruling', 'no ruling???', 'not determined','procedurally defaulted', 'time-barred (did not determine)']


data['new_outs'] = 0
drop = []
for index, x in data.iterrows():
	if x['misconduct_determination'] in misconduct_terms:
		data['new_outs'].iloc[index] = 0
	elif x['misconduct_determination'] in remove_terms:
		drop.append(index)
	else:
		data['new_outs'].iloc[index] = 1


nulls = pd.isna(data['misconduct_determination'])
to_drop = []
for x in range(len(nulls)):
	if nulls[x] == True:
		to_drop.append(x)

data = data.drop(drop)
data = data.drop(to_drop)
data = data.reset_index()
data['outcome'] = data['new_outs']

files = data[['txt_file', 'txt_link']].values.tolist()
texts = []

for index, row in data.iterrows():
	r = requests.get(row['txt_link'])
	file = (r.text)
	texts.append(file)


data['Text_files'] = texts
df = data[['orig_pdf', 'Text_files', 'outcome']]
df['aleg_terms'] = data[['allegation_term_1', 'allegation_term_2', 'allegation_term_3', 'allegation_term_4', 'allegation_term_5', 'allegation_term_6']].values.tolist()
df['deter_terms'] = data[['misconduct_determination_term1', 'misconduct_determination_term2', 'misconduct_determination_term3', 'misconduct_determination_term4', 'misconduct_determination_term5', 'misconduct_determination_term6']].values.tolist()
df['comb'] = ([a + b for a, b in zip(df['aleg_terms'], df['deter_terms'])])
print(df.keys())


c:\users\calla\appdata\local\programs\python\python36\lib\site-packages\pandas\core\indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


Index(['orig_pdf', 'Text_files', 'outcome', 'aleg_terms', 'deter_terms',
       'comb'],
      dtype='object')


c:\users\calla\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
c:\users\calla\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
c:\users\calla\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats

In [88]:
#all_terms = df['comb'].values.tolist()
df['comb'] = ([a + b for a, b in zip(df['aleg_terms'], df['deter_terms'])])
terms = [[term for term in sublist] for sublist in df['comb'].values]
terms = [term for lis in terms for term in lis]
terms = [term.replace("\n", " ").replace(".", "") for term in terms if type(term) != float]
terms = [term.lower() for term in terms]
terms = np.unique(terms)

df['comb'] = [[str(val) for val in sublist] for sublist in df['comb'].values]
df['comb'] = [' '.join(val) for val in df['comb'].values]

from sklearn.feature_extraction.text import CountVectorizer
vec = CountVectorizer(vocabulary = terms)
count3 = vec.fit_transform(df['Text_files'].values.tolist())
count3 = pd.DataFrame(count3.toarray(), columns=vec.get_feature_names())
cvec3 = count3.values.tolist()

sums = [sum(cvec) for cvec in cvec3]


c:\users\calla\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
c:\users\calla\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
c:\users\calla\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = 

In [87]:
X_train = cvec3[0:275]
y_train = df['outcome'].values.tolist()[0:275]
X_test = cvec3[275:]
y_test = df['outcome'].values.tolist()[275:]

from sklearn.linear_model import LogisticRegressionCV
clf = LogisticRegressionCV(random_state=0, solver='lbfgs', multi_class='multinomial').fit(X_train, y_train)
preds = clf.predict(X_test)
probs = clf.predict_proba(X_test)
score = clf.score(X_test, y_test)
print(score)


0.8021978021978022
